In [1]:
import numpy as np
import os 
import argparse
import cmath
import random
from numpy.linalg import norm
from numpy.linalg import inv
from numpy import matmul, transpose
j = complex(0,1)

Y_SIZE = 512
H_SIZE = 32
SIG1 = np.sqrt(0.001)
SIG2 = np.sqrt(0.1)
LAMBDA = 0.2

In [2]:
# Constructing X and F
x = np.random.choice([1+1j,1-1j,-1+1j,-1-1j],size=Y_SIZE)
X = np.diag(x,k=0)
F = np.empty((Y_SIZE,H_SIZE),dtype=np.csingle)
for i in range(F.shape[0]):
    for k in range(F.shape[1]):
        F[i][k] = cmath.exp(1j*(2*cmath.pi*i*k/512))        

# Noise vectors of different variances
# n1 = np.random.normal(loc=0, scale=SIG1,size=(Y_SIZE,2)).view(np.complex128)
# n2 = np.random.normal(loc=0, scale=SIG2,size=(Y_SIZE,2)).view(np.complex128)

# Constructing h
p = np.asarray([np.exp(-1*(LAMBDA)*i) for i in range(H_SIZE)])
p = p/np.sum(p**2)
a = np.random.normal(scale=0.5,size=H_SIZE)
b = np.random.normal(scale=0.5,size=H_SIZE)

# Reshape added else y was taking size 512*512
h = np.multiply(p,a+b*j).reshape(-1,1)

# Calculate A, where A = XF and y = XFh + n
A = matmul(X,F)

In [3]:
def pinv(A, alpha=0):
    Ah = np.asmatrix(A).getH()
    return  matmul(inv(matmul(Ah,A)+alpha*np.eye(np.shape(Ah)[0])),Ah)

In [4]:
def normalized_difference(y_original, y_hat):
    return norm(y_original-y_hat)/norm(y_original)

In [5]:
# Q1
def estimate_vanilla_h(A=A,h=h,sig=SIG1):
    n = np.random.normal(loc=0, scale=sig,size=(Y_SIZE,2)).view(np.complex128)
    y = matmul(A,h) + n
    h_hat = matmul(pinv(A), y)
    d = normalized_difference(h,h_hat)
    print(f"[Vanilla] Normalized difference for variance {np.round(sig**2, decimals=3)} : {d}")
    return h_hat,d
    
estimate_vanilla_h(sig=SIG1)
estimate_vanilla_h(sig=SIG2)

[Vanilla] Normalized difference for variance 0.001 : 0.023579829122856784
[Vanilla] Normalized difference for variance 0.1 : 0.24201371420473952


(matrix([[-1.61903064e-01+0.01131802j],
         [-1.41203539e-01+0.09794021j],
         [ 1.43414469e-01+0.04707481j],
         [ 4.13382887e-02-0.00337092j],
         [ 5.83107412e-02+0.05893105j],
         [-4.47588750e-02-0.09894171j],
         [ 1.47030616e-02-0.01033885j],
         [ 3.80519701e-02+0.04309481j],
         [ 2.74897115e-02-0.04858361j],
         [ 7.29310690e-03-0.00564459j],
         [-3.97814661e-02-0.04348208j],
         [-6.09242571e-03-0.03229589j],
         [ 1.12253821e-02+0.00576209j],
         [ 1.60277055e-02-0.00375089j],
         [ 5.29172535e-03-0.01145392j],
         [ 1.37344671e-03-0.01103197j],
         [ 1.91636542e-02+0.00518425j],
         [ 6.74849650e-04-0.01390343j],
         [ 6.47619869e-03+0.01066738j],
         [ 3.72694787e-03-0.00555499j],
         [ 2.68608913e-02-0.0001334j ],
         [ 6.85078563e-03-0.00578003j],
         [ 7.85040485e-03-0.00354006j],
         [ 1.19812525e-02-0.00362182j],
         [-8.17254201e-03-0.00125351j],


In [6]:
### Q2
### Sparse h
# h_sparse = h.copy()

# sparsity_points = [i for i in random.sample(range(0,H_SIZE),H_SIZE-6)]
# for i in sparsity_points:
#     h_sparse[i] = 0

# y1_sparse = np.matmul(A,h_sparse) + n1
# y2_sparse = np.matmul(A,h_sparse) + n2

# constraints_matrix = np.zeros((H_SIZE-6, H_SIZE))
# for i in range(len(sparsity_points)):
#     constraints_matrix[i,sparsity_points[i]]=1

# h_hat1_unconstrained = matmul(pinv(A), y1_sparse)
# Ah = np.asmatrix(A).getH()
# lambda_1 = matmul(2*inv(matmul(matmul(constraints_matrix, inv(matmul(Ah, A))),transpose(constraints_matrix))), matmul(constraints_matrix, h_hat1_unconstrained))
# h_hat1_constrained = h_hat1_unconstrained - 0.5*matmul(matmul(inv(matmul(Ah,A)), transpose(constraints_matrix)), lambda_1)
# d1_constrained = normalized_difference(h_sparse,h_hat1_constrained)

# h_hat2_unconstrained = matmul(pinv(A), y2_sparse)
# Ah = np.asmatrix(A).getH()
# lambda_2 = matmul(2*inv(matmul(matmul(constraints_matrix, inv(matmul(Ah, A))),transpose(constraints_matrix))), matmul(constraints_matrix, h_hat2_unconstrained))
# h_hat2_constrained = h_hat2_unconstrained - 0.5*matmul(matmul(inv(matmul(Ah,A)), transpose(constraints_matrix)), lambda_2)
# d2_constrained = normalized_difference(h_sparse,h_hat2_constrained)

# print(f"Normalized difference for variance {np.round(SIG1**2, decimals=3)} : {d1_constrained}")
# print(f"Normalized difference for variance {np.round(SIG2**2, decimals=1)} : {d2_constrained}")

In [7]:
# Q2
def estimate_sparse_h(A=A,h=h,sig=SIG1,sparsity_points=None):
    h_sparse = h.copy()
    
    if sparsity_points is None:
        sparsity_points = [i for i in random.sample(range(0,H_SIZE),H_SIZE-6)]
    for i in sparsity_points:
        h_sparse[i] = 0
    n = np.random.normal(loc=0, scale=sig,size=(Y_SIZE,2)).view(np.complex128)
    y_sparse = np.matmul(A,h_sparse) + n
    
    constraints_matrix = np.zeros((H_SIZE-6, H_SIZE))
    for i in range(len(sparsity_points)):
        constraints_matrix[i,sparsity_points[i]]=1

    h_hat_unconstrained = matmul(pinv(A), y_sparse)
    Ah = np.asmatrix(A).getH()
    
    lambda_ = matmul(2*inv(matmul(matmul(constraints_matrix, inv(matmul(Ah, A))), \
                                  transpose(constraints_matrix))), matmul(constraints_matrix, h_hat_unconstrained))
    h_hat_constrained = h_hat_unconstrained - 0.5*matmul(matmul(inv(matmul(Ah,A)), \
                                                                transpose(constraints_matrix)), lambda_)
    d_constrained = normalized_difference(h_sparse,h_hat_constrained)

    print(f"[Sparse] Normalized difference for variance {np.round(sig**2, decimals=3)} : {d_constrained}")
    return h_hat_constrained,d_constrained

estimate_sparse_h(sig=SIG1)
estimate_sparse_h(sig=SIG2)

[Sparse] Normalized difference for variance 0.001 : 0.026693638248561545
[Sparse] Normalized difference for variance 0.1 : 0.2426883205893068


(matrix([[-1.50251592e-01+8.48751270e-03j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00+1.73472348e-18j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 1.24926266e-02-1.40023102e-02j],
         [ 0.00000000e+00+1.08420217e-19j],
         [ 0.00000000e+00+8.67361738e-19j],
         [ 0.00000000e+00-4.33680869e-19j],
         [ 0.00000000e+00+4.33680869e-19j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00+5.42101086e-20j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00-1.73472348e-18j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 3.46944695e-18-2.16840434e-19j],
         [-4.33680869e-19+0.00000000e+00j],
         [-8.42816204e-03-1.94405270e-02j],
         [-1.73472348e-18+0.00000000e+00j],
         [ 0.00000000e+00+0.00000000e+00j],
         [-8.67361738e-19+0.0000

In [8]:
# Q3

# # Set guard bands of width 180 on either side
# X_ = X.copy()
# X_[:180] = 0
# X_[-180:] = 0
# A_ = matmul(X_,F)

# y1_ = matmul(A_,h) + n1
# y2_ = matmul(A_,h) + n2

# # define alpha values
# alpha1 = 1
# alpha2 = 1

# ## LSE
# h_hat1_ = matmul(pinv(A_, alpha1), y1_)
# h_hat2_ = matmul(pinv(A_, alpha2), y2_)
# d1_ = normalized_difference(h,h_hat1_)
# d2_ = normalized_difference(h,h_hat2_)

# # print("h vector was : ",h)
# # print("h_hat1 vector is :",h_hat1)
# # print("h_hat2 vector is :",h_hat2)
# print(f"Normalized difference for variance {np.round(SIG1**2, decimals=3)} and alpha {alpha1} : {d1}")
# print(f"Normalized difference for variance {np.round(SIG2**2, decimals=1)} and alpha {alpha2} : {d2}")

In [9]:
# Q3a
X_ = X.copy()
X_[:180] = 0
X_[-180:] = 0
A_ = matmul(X_,F)

def estimate_guard_h_no_reg(A=A_,h=h,sig=SIG1):
    h_vanilla,d_vanilla = estimate_vanilla_h(A=A,sig=sig)
    h_sparse,d_sparse = estimate_sparse_h(A=A,sig=sig)
    return h_vanilla, h_sparse, d_vanilla, d_sparse

estimate_guard_h_no_reg(sig=SIG1)
estimate_guard_h_no_reg(sig=SIG2)

# Q3b
def estimate_guard_h_reg(A=A_,h=h,alpha=1,sig=SIG1):
    n = np.random.normal(loc=0, scale=sig,size=(Y_SIZE,2)).view(np.complex128)
    y_ = matmul(A,h) + n
    
    h_hat_ = matmul(pinv(A_, alpha), y_)
    d_ = normalized_difference(h,h_hat_)
    
    print(f"[Regularized] Normalized difference for variance {np.round(sig**2, decimals=3)} and alpha {alpha} : {d_}")
    return h_hat_,d_
    
estimate_guard_h_reg(alpha=0.01,sig=SIG1)
estimate_guard_h_reg(alpha=1,sig=SIG1)

[Vanilla] Normalized difference for variance 0.001 : 47260060.993050605
[Sparse] Normalized difference for variance 0.001 : 60988715.398452155
[Vanilla] Normalized difference for variance 0.1 : 123192884.3198801
[Sparse] Normalized difference for variance 0.1 : 649448819.9509047
[Regularized] Normalized difference for variance 0.001 and alpha 0.01 : 0.8165667501475998
[Regularized] Normalized difference for variance 0.001 and alpha 1 : 0.6256980024571146


(matrix([[-0.11258671+0.00783088j],
         [-0.09403284+0.03484535j],
         [ 0.12907477-0.00757026j],
         [-0.07519321-0.04134948j],
         [ 0.01899608+0.06552031j],
         [-0.00149228-0.04815789j],
         [ 0.01302032+0.00585166j],
         [-0.02298045+0.03066243j],
         [ 0.01358778-0.04115866j],
         [ 0.00802124+0.02749295j],
         [-0.02244336-0.00664417j],
         [ 0.01861976-0.00540079j],
         [-0.00297431+0.00470389j],
         [-0.00880297+0.00172989j],
         [ 0.00720157-0.00513535j],
         [ 0.00337164+0.00269602j],
         [-0.01062004+0.00215647j],
         [ 0.00703536-0.00496199j],
         [ 0.00327319+0.00467597j],
         [-0.00957866-0.0033533j ],
         [ 0.00574335+0.00271054j],
         [ 0.00366709-0.00211629j],
         [-0.00832845+0.00017165j],
         [ 0.00307808+0.00253503j],
         [ 0.00638604-0.00340973j],
         [-0.00930261+0.00110599j],
         [ 0.00179699+0.00191515j],
         [ 0.00755313-0.0016

In [10]:
# Q4

# Building constraints matrix
number_of_constraints = 3
constraints_matrix = np.zeros((number_of_constraints, H_SIZE))
constraints_matrix[0,0]=1
constraints_matrix[0,1]=-1
constraints_matrix[1,2]=1
constraints_matrix[1,3]=-1
constraints_matrix[2,4]=1
constraints_matrix[2,5]=-1

def estimate_constrained_h(A=A,h=h,sig=SIG1):
    h_constrained = h.copy()
    h_constrained[0] = h_constrained[1]
    h_constrained[2] = h_constrained[3]
    h_constrained[4] = h_constrained[5]
    
    n = np.random.normal(loc=0, scale=sig,size=(Y_SIZE,2)).view(np.complex128)
    y_constrained = np.matmul(A,h_constrained) + n

    h_hat_unconstrained = matmul(pinv(A), y_constrained)
    Ah = np.asmatrix(A).getH()
    lambda_ = matmul(2*inv(matmul(matmul(constraints_matrix, inv(matmul(Ah, A))),\
                                  transpose(constraints_matrix))), matmul(constraints_matrix, h_hat_unconstrained))
    h_hat_constrained = h_hat_unconstrained - 0.5*matmul(matmul(inv(matmul(Ah,A)),\
                                                                transpose(constraints_matrix)), lambda_)
    d_constrained = normalized_difference(h_constrained,h_hat_constrained)
    print(f"[Constrained] Normalized difference for variance {np.round(sig**2, decimals=3)} : {d_constrained}")
    return h_hat_constrained,d_constrained

estimate_constrained_h(sig=SIG1)
estimate_constrained_h(sig=SIG2)

[Constrained] Normalized difference for variance 0.001 : 0.024363821477191707
[Constrained] Normalized difference for variance 0.1 : 0.26025496008323984


(matrix([[-0.13237759+0.09788829j],
         [-0.13237759+0.09788829j],
         [ 0.05957897-0.01742272j],
         [ 0.05957897-0.01742272j],
         [-0.02340432-0.0924025j ],
         [-0.02340432-0.0924025j ],
         [ 0.01382686-0.00672852j],
         [ 0.02350408+0.03152161j],
         [ 0.02820527-0.07113179j],
         [ 0.0273901 -0.00453935j],
         [-0.03046512-0.02221011j],
         [ 0.00874176-0.03184686j],
         [ 0.02145266-0.01938978j],
         [ 0.01554522+0.01383943j],
         [-0.00139744-0.00475811j],
         [-0.00168972-0.00288233j],
         [-0.00033072-0.00155752j],
         [ 0.00173731-0.02562347j],
         [-0.00633767+0.00737881j],
         [ 0.00685564+0.00344266j],
         [ 0.01919747-0.01331766j],
         [ 0.01111656+0.00894428j],
         [-0.01391023+0.01065036j],
         [-0.00963073-0.00312001j],
         [ 0.01278015-0.00724042j],
         [-0.00747396-0.02005627j],
         [ 0.00786309-0.01006674j],
         [ 0.00767993+0.0016

In [11]:
# Q5
Somp = []
n = np.random.normal(loc=0, scale=SIG2,size=(Y_SIZE,2)).view(np.complex128)
y = matmul(A,h) + n
r = y.copy()
ko = 6
Ah = np.asmatrix(A).getH()
for k in range(ko):
    t_ = np.matmul(Ah,r)
    t = np.argmax(np.abs(np.matmul(Ah,r)))
    Somp.append(int(t))
    p = np.matmul(A[:,Somp],pinv(A[:,Somp]))
    r = np.matmul((np.eye(Y_SIZE) - p),y) 

sparsity_points = [x for x in range(H_SIZE) if x not in Somp]
estimate_sparse_h(sig=SIG2,sparsity_points=sparsity_points)

[Sparse] Normalized difference for variance 0.1 : 0.07941798286762564


(matrix([[-1.38848468e-01+2.13595317e-02j],
         [-1.47036660e-01+9.20469445e-02j],
         [ 1.61416246e-01+7.71074849e-02j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 6.92217113e-02+5.30732458e-02j],
         [-3.24060404e-02-9.65576823e-02j],
         [ 0.00000000e+00+0.00000000e+00j],
         [-2.16840434e-19+0.00000000e+00j],
         [ 4.65188115e-02-5.23655870e-02j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00+0.00000000e+00j],
         [-6.93889390e-18-1.73472348e-18j],
         [-3.46944695e-18+0.00000000e+00j],
         [ 0.00000000e+00+1.73472348e-18j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 8.67361738e-19+0.00000000e+00j],
         [ 0.00000000e+00-4.33680869e-19j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 1.73472348e-18-1.73472348e-18j],
         [ 0.00000000e+00+8.67361738e-19j],
         [ 0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00-5.4210